In [ ]:
#libraries
import pandas as pd
import numpy as np

In [ ]:
#candiadate info dataset
cand_kipa = pd.read_sas('/home/kidneyData/DATA_SP_2022/cand_kipa.sas7bdat')

In [ ]:
cand_kipa.head()

In [ ]:
cand_kipa.shape

In [ ]:
cand_kipa.columns

In [ ]:
tx_ki = pd.read_sas('/home/kidneyData/DATA_SP_2022/tx_ki.sas7bdat')

In [ ]:
tx_ki = pd.read_sas('/home/kidneyData/DATA_SP_2022/tx_ki.sas7bdat')

In [ ]:
tx_ki.columns

In [ ]:
tx_ki.shape

In [ ]:
# donor_deceased
donor_deceased = pd.read_sas('/home/kidneyData/DATA_SP_2022/donor_deceased.sas7bdat')

In [ ]:
# donot_live.head
donor_deceased.shape

In [ ]:
donor_deceased.columns

In [ ]:
# Finding Common columns of donor_living and tx_ki
common_columns = np.intersect1d(donor_deceased.columns, tx_ki.columns)
print(common_columns)

In [ ]:
 tx_donor_deceased = tx_ki.merge(donor_deceased, left_index=False, right_index=False,
             on='DONOR_ID', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [ ]:
tx_donor_deceased.shape

In [ ]:
# Merging the above merged donor_living and tx_ki now to cand_kipa 

In [ ]:
# Finding Common columns of tx_donor_live and cand_kipa using numpy
common_columns = np.intersect1d(tx_donor_deceased.columns, cand_kipa.columns)
print(common_columns)

In [ ]:
cand_tx_donor_deceased = cand_kipa.merge(tx_donor_deceased, left_index=False, right_index=False,
             on='PX_ID', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [ ]:
 cand_tx_donor_deceased.shape

In [ ]:
# rec_histo
rec_histo = pd.read_sas('/home/kidneyData/DATA_SP_2022/rec_histo.sas7bdat')

In [ ]:
rec_histo.shape

In [ ]:
 # Finding Common columns of tx_donor_live and cand_kipa using numpy
common_columns = np.intersect1d(cand_tx_donor_deceased.columns, rec_histo.columns)
print(common_columns)


In [ ]:
 cand_tx_donor_deceased_rec = cand_tx_donor_deceased.merge(rec_histo, left_index=False, right_index=False,
             on='REC_HISTO_TX_ID', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [ ]:
cand_tx_donor_deceased_rec.shape

In [ ]:
cand_tx_donor_deceased_rec.head()

In [ ]:
 cand_tx_donor_deceased_rec.to_csv("cand_tx_donor_deceased_rec.csv")

In [ ]:
dataset_deceased=pd.read_csv('cand_tx_donor_deceased_rec.csv', low_memory=False)
dataset_deceased.head()

In [ ]:
#  dataset_deceased=pd.read_csv('cand_tx_donor_deceased_rec.csv', low_memory=False)
# dataset_deceased.head()
# dataset_deceased.drop(dataset_deceased.columns[[0]], axis = 1, inplace = True)

In [ ]:
dataset_deceased.shape

### IMMUNOGENICITY AND EPLET LOAD CALCULATIONS

In [ ]:
dataset_deceased=pd.read_csv('cand_tx_donor_deceased_rec.csv', low_memory=False)
dataset_deceased.head()
dataset_deceased.drop(dataset_deceased.columns[[0]], axis = 1, inplace = True)

In [ ]:
dataset_deceased.shape

In [ ]:
import os, sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent) + "/ImmunogenicityUtilities(2)/")

In [ ]:
pip install swifter

In [ ]:
import pandas as pd
import ImmunogenicityUtilities as IU
import EpletUtilities as EU
import swifter

In [ ]:
alleleTypeList_dict = ["A","B","DRB1"]
alleleTypeList = ["A","B", "DR"]
allele_conversion_dict = IU.make_allele_conversion_dict('immunogenicity-utilities/Dictionaries/HLAConversionDataDictionary.csv')
allele_mapping_dict = IU.make_allele_mapping_dict('immunogenicity-utilities/Dictionaries/HLAConversionDataDictionary.csv')
master_dict = IU.make_haplo_frequency_dict("immunogenicity-utilities/Dictionaries/freq_dic/",alleleTypeList_dict)

In [ ]:
dataset_deceased=pd.read_csv('cand_tx_donor_deceased_rec.csv', encoding="latin1", low_memory=False)
dataset_deceased.drop(dataset_deceased.columns[[0]], axis = 1, inplace = True)
dataset_deceased.head()

In [ ]:
 dataset_deceased.shape

In [ ]:
dataset_deceased = dataset_deceased[[
 'DON_RACE',
 'DON_A1',
 'DON_A2',
 'DON_B1',
 'DON_B2',
 'DON_DR1',
 'DON_DR2',
 'PERS_ID',
 'CAN_RACE',
 'REC_A1',
 'REC_A2',
 'REC_B1',
 'REC_B2',
 'REC_DR1',
 'REC_DR2']].dropna().reset_index(drop=True)

In [ ]:
dataset_deceased.head(5)

In [ ]:
def race_label(race):
    if race == 8:
        return "White"
    elif race == 16:
        return "Black or African American"
    elif race == 32:
        return "American Indian or Alaska Native"
    elif race == 64:
        return "Asian"
    elif race == 128:
        return "Native Hawaiian or Other Pacific Islander"
    elif race == 256:
        return "Arab or Middle Eastern"
    elif race == 512:
        return "Indian Sub-continent"
    elif race == 1024:
        return "Unknown (for Donor Referral only)"
    elif race == 2000:
        return "Hispanic/Latino"
    else:
        return "Multi-Racial" 

In [ ]:
# Apply race_label function to DON_RACE column
dataset_deceased['DON_RACE'] =  dataset_deceased['DON_RACE'].apply(race_label)

# Apply race_label function to CAN_RACE column
dataset_deceased['CAN_RACE'] =  dataset_deceased['CAN_RACE'].apply(race_label)

In [ ]:
 dataset_deceased.head(5)

### HLA Conversion to High Resolution

In [ ]:
dataset_deceased[["DON_A1_HR","DON_A2_HR","DON_B1_HR","DON_B2_HR","DON_DR1_HR","DON_DR2_HR"]] = dataset_deceased.swifter.apply(lambda row: IU.get_high_resolution(row[0],row[1:7],alleleTypeList,allele_mapping_dict,allele_conversion_dict,master_dict),axis=1,result_type="expand") 

In [ ]:
dataset_deceased.head(5)

In [ ]:
#convert all receipient HLA to high resolution
dataset_deceased[["REC_A1_HR","REC_A2_HR","REC_B1_HR","REC_B2_HR","REC_DR1_HR","REC_DR2_HR"]] = dataset_deceased.swifter.apply(lambda row: IU.get_high_resolution(row[8],row[9:15],alleleTypeList,allele_mapping_dict,allele_conversion_dict,master_dict),axis=1,result_type="expand") 

In [ ]:
dataset_deceased.head(5)

In [ ]:
dataset_deceased.shape

In [ ]:
dataset_deceased.isna().sum()

In [ ]:
dataset_deceased = dataset_deceased.dropna(axis=0, subset=['DON_A1_HR'])
dataset_deceased = dataset_deceased.dropna(axis=0, subset=['REC_A1_HR'])

In [ ]:
 dataset_deceased.shape

In [ ]:
aa_df = pd.read_csv("AA_Code.csv")
aa_dict = IU.make_amino_acid_dict(aa_df)
classIseqdf = pd.read_csv("immunogenicity-utilities/Dictionaries/Aligned_Seq.csv")
classIseq = IU.make_aligned_seq_dict(classIseqdf,aa_dict)
classIIseq = IU.make_allele_sequence_dict(['DPA1', 'DPB1', 'DQA1', 'DRB1345', 'DQB1'],"immunogenicity-utilities/Dictionaries/Class_Two_Seq/",aa_dict)

In [ ]:
 dataset_deceased.isna().sum()

In [ ]:
dataset_deceased.iloc[0,15:19]

In [ ]:
 dataset_deceased.iloc[0,21:25]

In [ ]:
dataset_deceased[["AB_HMS","AB_EMS","AB_AMS"]] = pd.DataFrame(list(dataset_deceased.swifter.apply(lambda row: IU.typeICalcs(row[15:19],row[21:25],classIseq),axis=1)))

In [ ]:
dataset_deceased.iloc[0,19:21]

In [ ]:
dataset_deceased.iloc[0,25:27]

In [ ]:
dataset_deceased[['DR_HMS', 'DR_EMS', 'DR_AMS', 'DR_alphaHMS', 'DR_alphaEMS', 'DR_alphaAMS', 'DR_betaHMS',
       'DR_betaEMS', 'DR_betaAMS']] = pd.DataFrame(list(dataset_deceased.swifter.apply(lambda row: IU.typeIICalcs(row[19:21],row[25:27],classIIseq),axis=1)))

In [ ]:
dataset_deceased.head()

In [ ]:
dataset_deceased['Avg_EMS'] = dataset_deceased[['AB_EMS', 'DR_EMS']].mean(axis=1)
dataset_deceased['Avg_HMS'] = dataset_deceased[['AB_HMS', 'DR_HMS']].mean(axis=1)
dataset_deceased['Avg_AMS'] = dataset_deceased[['AB_AMS', 'DR_AMS']].mean(axis=1)

In [ ]:
 dataset_deceased.head()

In [ ]:
dataset_deceased.isna().sum()

### EPLET LOAD CALCULATIONS

In [ ]:
melt_Df = pd.read_csv("immunogenicity-utilities/Dictionaries/Eplet_Allele_ALL_Drop_NA_Melt.csv")
eplet_dict, eplet_type_dict = EU.build_dictionaries(melt_Df)

In [ ]:
dataset_deceased[["DON_EPLETS","REC_EPLETS"]] = pd.DataFrame(list(dataset_deceased.swifter.apply(lambda row: EU.compare_eplets(row[['DON_A1_HR', 'DON_A2_HR', 'DON_B1_HR', 'DON_B2_HR', 'DON_DR1_HR',
       'DON_DR2_HR']],row[['REC_A1_HR',
       'REC_A2_HR', 'REC_B1_HR', 'REC_B2_HR', 'REC_DR1_HR', 'REC_DR2_HR']],eplet_dict)  ,axis=1)))

In [ ]:
 dataset_deceased.head()

In [ ]:
dataset_deceased = dataset_deceased.dropna(axis=0, subset=['AB_HMS'])

In [ ]:
dataset_deceased.head()

In [ ]:
 dataset_deceased.to_csv("dataset_deceased_immuno_eplet.csv")

### SURVIVAL ANALYSIS

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
 #transplant info dataset
tx_ki = pd.read_sas('/home/kidneyData/DATA_SP_2022/tx_ki.sas7bdat')

In [ ]:
print(tx_ki.columns)

In [ ]:
tx_ki = tx_ki[[
 'PERS_ID',
 'CAN_GENDER',
 'REC_AGE_AT_TX',
 'TFL_GRAFT_DT',
 'REC_FAIL_DT',
 'PERS_RELIST',
 'PERS_RETX',
 'REC_RESUM_MAINT_DIAL_DT',
 'PERS_SSA_DEATH_DT',
 'PERS_OPTN_DEATH_DT',
 #'PERS_RESTRICT_DEATH_DT',
 'TFL_DEATH_DT',
 'TFL_LAFUDATE',
 'TFL_ENDTXFU',
 'REC_TX_DT']]

### PATIENT SURVIVED - PS

In [ ]:
#PS = “Died”, if there is a record in any of the following variables 
pd.options.mode.chained_assignment = None
tx_ki['PS_PS'] = np.where((np.isnat(tx_ki.PERS_SSA_DEATH_DT)), '0', '1')
tx_ki['PS_PO'] = np.where((np.isnat(tx_ki.PERS_OPTN_DEATH_DT)), '0', '1')
#tx_ki['PS_PR'] = np.where((np.isnat(tx_ki.PERS_RESTRICT_DEATH_DT)), '0', '1')
tx_ki['PS_PD'] = np.where((np.isnat(tx_ki.TFL_DEATH_DT)), '0', '1')

#PS column 
#tx_ki['PS'] = tx_ki[['PS_PS', 'PS_PO', 'PS_PR', 'PS_PD']].max(axis=1)
tx_ki['PS'] = tx_ki[['PS_PS', 'PS_PO', 'PS_PD']].max(axis=1)

In [ ]:
 tx_ki = tx_ki[[
 'PERS_ID',
 'CAN_GENDER',
 'REC_AGE_AT_TX',
 'TFL_GRAFT_DT',
 'REC_FAIL_DT',
 'PERS_RELIST',
 'PERS_RETX',
 'REC_RESUM_MAINT_DIAL_DT',
 'PERS_SSA_DEATH_DT',
 'PERS_OPTN_DEATH_DT',
 #'PERS_RESTRICT_DEATH_DT',
 'TFL_DEATH_DT',
 'TFL_LAFUDATE',
 'TFL_ENDTXFU',
 'REC_TX_DT',
 'PS']]

In [ ]:
 tx_ki.head()

### GRAFT LOSS DUE TO KIDNEY FAILURE

In [ ]:
#KGL = “Failed”, if there is a record in any of the following variables 
pd.options.mode.chained_assignment = None 
tx_ki['KGL_G'] = np.where((np.isnat(tx_ki.TFL_GRAFT_DT)), '0', '1')
tx_ki['KGL_F'] = np.where((np.isnat(tx_ki.REC_FAIL_DT)), '0', '1')
tx_ki['KGL_P'] = np.where((np.isnat(tx_ki.PERS_RELIST)), '0', '1')
tx_ki['KGL_RR'] = np.where((np.isnat(tx_ki.REC_RESUM_MAINT_DIAL_DT)), '0', '1')
tx_ki['KGL_RF'] = np.where((np.isnat(tx_ki.REC_FAIL_DT)), '0', '1')

#KGL column
tx_ki['KGL'] = tx_ki[['KGL_G', 'KGL_F', 'KGL_P', 'KGL_RR', 'KGL_RF']].max(axis=1)

In [ ]:
tx_ki = tx_ki[[
 'PERS_ID',
 'CAN_GENDER',
 'REC_AGE_AT_TX',
 'TFL_GRAFT_DT',
 'REC_FAIL_DT',
 'PERS_RELIST',
 'PERS_RETX',
 'REC_RESUM_MAINT_DIAL_DT',
 'PERS_SSA_DEATH_DT',
 'PERS_OPTN_DEATH_DT',
 #'PERS_RESTRICT_DEATH_DT',
 'TFL_DEATH_DT',
 'TFL_LAFUDATE',
 'TFL_ENDTXFU',
 'REC_TX_DT',
 'PS',
 'KGL']]

In [ ]:
tx_ki.head()

### DEATH CENSORED GRAFT LOSS - DCGL
#### GRAFT LOSS DUE TO DEATH

In [ ]:
tx_ki['DCGL'] = np.where((tx_ki.PS == 1) & (tx_ki.KGL == 0), 1,0)

In [ ]:
 tx_ki.head()

### GRAFT LOSS - GL

In [ ]:
print(tx_ki[['KGL', 'DCGL']].dtypes)

In [ ]:
tx_ki[['KGL', 'DCGL']] = tx_ki[['KGL', 'DCGL']].apply(pd.to_numeric, errors='coerce')

In [ ]:
tx_ki[['KGL', 'DCGL']] = tx_ki[['KGL', 'DCGL']].fillna(0)

In [ ]:
tx_ki['GL'] = tx_ki[['KGL', 'DCGL']].max(axis=1)

In [ ]:
tx_ki.head()

In [ ]:
tx_ki.isna().sum()

### NUMBER OF DAYS UNTIL EVENT OCCURED - DAYS

In [ ]:
 #minimum date of TFL_LAFUDATE, TFL_ENDTXFU
tx_ki['TFL_LAF_END_MIN'] = tx_ki[['TFL_LAFUDATE', 'TFL_ENDTXFU']].min(axis=1)

In [ ]:
tx_ki['DAYS'] = np.where(tx_ki.PS == 0, tx_ki.TFL_LAFUDATE - tx_ki.REC_TX_DT, tx_ki.TFL_LAF_END_MIN - tx_ki.REC_TX_DT)
tx_ki['DAYS']=(tx_ki['DAYS']).dt.days

In [ ]:
tx_ki = tx_ki[[
 'PERS_ID',
 'CAN_GENDER',
 'REC_AGE_AT_TX',
 'TFL_GRAFT_DT',
 'REC_FAIL_DT',
 'PERS_RELIST',
 'PERS_RETX',
 'REC_RESUM_MAINT_DIAL_DT',
 'PERS_SSA_DEATH_DT',
 'PERS_OPTN_DEATH_DT',
# 'PERS_RESTRICT_DEATH_DT',
 'TFL_DEATH_DT',
 'TFL_LAFUDATE',
 'TFL_ENDTXFU',
 'REC_TX_DT',
 'PS',
 'KGL',
 'DCGL',
 'GL',
 'DAYS']]

In [ ]:
tx_ki.head()

### MERGING WITH DATASET FROM IMMUNOGENICITY AND EPLETS CALCULATIONS

In [ ]:
dataset_survival = pd.read_csv('dataset_deceased_immuno_eplet.csv', encoding="latin1", low_memory=False)

In [ ]:
dataset_survival.shape

In [ ]:
 dataset_survival.head()

In [ ]:
 #merge survival dataset with immunogenicity and eplet load calcs
dataset_survival = dataset_survival.merge(tx_ki, left_index=False, right_index=False,
             on='PERS_ID', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [ ]:
dataset_survival.shape

In [ ]:
dataset_survival.drop(columns=dataset_survival.columns[0], axis=1, inplace=True)
dataset_survival.head()

In [ ]:
dataset_survival.isna().sum()

In [ ]:
dataset_survival.shape

In [ ]:
#save data
dataset_survival.to_csv("dataset_survival_deceased.csv")

### KAPLAN MEIER ANALYSIS

In [ ]:
#libraries
import pandas as pd
import numpy as np

In [ ]:
pip install lifelines

In [ ]:
from lifelines import KaplanMeierFitter

In [ ]:
 #loading data
dataset_survival = pd.read_csv('dataset_survival_deceased.csv', encoding="latin1", low_memory=False)
dataset_survival.head()
#dataset_survival_living 

In [ ]:
dataset_survival.shape

In [ ]:
dataset_survival.drop(columns=dataset_survival.columns[0], axis=1, inplace=True)

In [ ]:
dataset_survival.shape

In [ ]:
dataset_survival.columns

### KAPLAN CURVE SHOWING OVERALL KIDNEY GRAFT ALLOCATION SURVIVAL

In [ ]:
 ## create a kmf object
kmf = KaplanMeierFitter()

T=dataset_survival['DAYS'] #duration
C=dataset_survival['GL'] # 0 = No Graft Loss, Graft Loss=1

kmf.fit(T, C,label='Kaplan Meier Estimate')

In [ ]:
#KPM curve of  Overall Kidney Graft Allocation with Confidence Interval  2000-2010
%pylab inline
figsize(12,6)
kmf.plot()
plt.xlabel('Time (Days)')
plt.ylabel('Survival probability estimate')
plt.title('Overall KPM survival curve among kidney transplant patients_Deceased Donors(in days)')

In [ ]:
#KPM curve of  Overall Kidney Graft Allocation without Confidence Interval
kmf.plot(ci_show=False) 
plt.xlabel('Time (Days)')
plt.ylabel('Survival probability estimate')
plt.title('Kaplan Meier Curve of Kidney Graft Survival')

In [ ]:
print("The median survival time :",kmf.median_survival_time_)

In [ ]:
 #the survival probabilities at time 1000, 2000, 3000, 3500, 4000, 5000, 6000, 7000 days etc
survivals = pd.DataFrame([1000, 2000, 3000, 3500, 3598, 4000, 5000, 6000, 7000, 8000,9000, 10000, 11000], columns = ['time'])
survivals.loc[:, 'Survival Prob'] = kmf.survival_function_at_times(survivals['time']).values
survivals

In [ ]:
#confidence interval
kmf.confidence_interval_

### SUBGROUP ANALYSIS
#### GENDER

In [ ]:
dataset_survival['CAN_GENDER'].unique()

In [ ]:
dataset_survival.CAN_GENDER.value_counts()

In [ ]:
male=(69655/(69655+45426))*100
female=(45426/(69655+45426))*100
print("The male percentage is :",male)
print("The female percentage is :",female)

In [ ]:
ax = subplot(111)

for r in dataset_survival['CAN_GENDER'].unique(): 
    ix = dataset_survival['CAN_GENDER'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by gender-Deceased Donors')

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.CAN_GENDER ==  "b'M'"]

S2 = dataset_survival[dataset_survival.CAN_GENDER ==  "b'F'"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

#### RACE

In [ ]:
 dataset_survival['CAN_RACE'].unique()

In [ ]:
dataset_survival.CAN_RACE.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['CAN_RACE'].unique(): 
    ix = dataset_survival['CAN_RACE'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by race - Deceased Donors(in days)')

In [ ]:
 #subdivision of kpm plot by whites, Black or African American and Asian 
ax = subplot(111)

for r in dataset_survival['CAN_RACE'].unique()[:3]: 
    ix = dataset_survival['CAN_RACE'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False) 

In [ ]:
 #subdivision kpm plot by hispanic/Latino, American Indian or Alaska Native, Native Hawaiian or Other Pacific Islander 
ax = subplot(111)

for r in dataset_survival['CAN_RACE'].unique()[3:]: 
    ix = dataset_survival['CAN_RACE'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.CAN_RACE ==  "White"]

S2 = dataset_survival[dataset_survival.CAN_RACE ==  "Black or African American"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.CAN_RACE ==  "White"]

S2 = dataset_survival[dataset_survival.CAN_RACE ==  "Asian"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.CAN_RACE ==  "Asian"]

S2 = dataset_survival[dataset_survival.CAN_RACE ==  "Black or African American"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

#### AGE

In [ ]:
plt.hist(dataset_survival['REC_AGE_AT_TX'])

In [ ]:
 #fxn to create age group
def age_group(age):
    if age <= 40:  
        return "< 40"
    elif age <= 60: 
        return "41 - 60"
    else:           
        return "> 60"

In [ ]:
 # Apply age_group function to REC_AGE_AT_TX column
dataset_survival['REC_AGE_AT_TX'] = dataset_survival['REC_AGE_AT_TX'].apply(age_group)

In [ ]:
 #age categories count
dataset_survival.REC_AGE_AT_TX.value_counts()

In [ ]:
#KPM Curve by age group
ax = subplot(111)

for r in dataset_survival['REC_AGE_AT_TX'].unique(): 
    ix = dataset_survival['REC_AGE_AT_TX'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by age groups - Deceased Donors (in days)')

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.REC_AGE_AT_TX ==  "< 40"]

S2 = dataset_survival[dataset_survival.REC_AGE_AT_TX ==  "> 60"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.REC_AGE_AT_TX ==  "41 - 60"]

S2 = dataset_survival[dataset_survival.REC_AGE_AT_TX ==  "> 60"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.REC_AGE_AT_TX ==  "< 40"]

S2 = dataset_survival[dataset_survival.REC_AGE_AT_TX ==  "41 - 60"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

#### IMMUNOGENICITY
##### Avg_EMS

In [ ]:
dataset_survival['Avg_EMS'].unique()

###### Cut off 1.8

In [ ]:
 #fxn to create age group (Using cut of 1.8 for this)
def Avg_EMS_group(Avg_EMS):
    if Avg_EMS <= 1.8: 
        return "<= 1.8"
    else:
        return ">1.8"

In [ ]:
 dataset_survival['Avg_EMS_group'] = dataset_survival['Avg_EMS'].apply(Avg_EMS_group)

In [ ]:
  dataset_survival.Avg_EMS_group.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group'].unique(): 
    ix = dataset_survival['Avg_EMS_group'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by Average EMS - Deceased Donors (in days)')

In [ ]:
 # Checking if the diffrence between survival of the two groups is statistically significant
#Log-Rank Test (test statistical difference between the two survival curves)
S1 = dataset_survival[dataset_survival.Avg_EMS_group ==  "<= 1.8"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group ==  ">1.8"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

###### Cut off 3.0

In [ ]:
#fxn to create age group (Using cut of 1.8 for this)
def Avg_EMS_group(Avg_EMS):
    if Avg_EMS <= 3.0: 
        return "<= 3.0"
    else:
        return ">3.0"

In [ ]:
dataset_survival['Avg_EMS_group2'] = dataset_survival['Avg_EMS'].apply(Avg_EMS_group)

In [ ]:
dataset_survival.Avg_EMS_group2.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group2'].unique(): 
    ix = dataset_survival['Avg_EMS_group2'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by Average EMS groups - Deceased Donors (in days)')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group2 ==  "<= 3.0"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group2 ==  ">3.0"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

###### Cut off 7.0

In [ ]:
 #fxn to create age group (Using cut of 1.8 for this)
def Avg_EMS_group(Avg_EMS):
    if Avg_EMS <= 7.0: 
        return "<= 7.0"
    else:
        return ">7.0"

In [ ]:
dataset_survival['Avg_EMS_group3'] = dataset_survival['Avg_EMS'].apply(Avg_EMS_group)

In [ ]:
dataset_survival.Avg_EMS_group3.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group3'].unique(): 
    ix = dataset_survival['Avg_EMS_group3'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by Average EMS groups - Deceased Donors (in days)')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group3 ==  "<= 7.0"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group3 ==  ">7.0"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

###### Cut off 1.1

In [ ]:
 #fxn to create age group (Using cut of 1.8 for this)
def Avg_EMS_group(Avg_EMS):
    if Avg_EMS <= 1.1: 
        return "<= 1.1"
    else:
        return ">1.1"

In [ ]:
dataset_survival['Avg_EMS_group4'] = dataset_survival['Avg_EMS'].apply(Avg_EMS_group)

In [ ]:
  dataset_survival.Avg_EMS_group4.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group4'].unique(): 
    ix = dataset_survival['Avg_EMS_group4'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients using average EMS cutoff of 1.1-Deceased Donors (in days)')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group4 ==  "<= 1.1"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group4 ==  ">1.1"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

###### Cut of 4.0

In [ ]:
#fxn to create age group (Using cut of 1.8 for this)
def Avg_EMS_group(Avg_EMS):
    if Avg_EMS <= 4.0: 
        return "<= 4.0"
    else:
        return ">4.0"

In [ ]:
dataset_survival['Avg_EMS_group5'] = dataset_survival['Avg_EMS'].apply(Avg_EMS_group)

In [ ]:
 dataset_survival.Avg_EMS_group5.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group5'].unique(): 
    ix = dataset_survival['Avg_EMS_group5'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients using average EMS cutoff of 4.0-Deceased Donors (in days)')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group5 ==  "<= 4.0"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group5 ==  ">4.0"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

###### Cut off 5.0

In [ ]:
#fxn to create age group (Using cut of 1.8 for this)
def Avg_EMS_group(Avg_EMS):
    if Avg_EMS <= 5.0: 
        return "<= 5.0"
    else:
        return ">5.0"

In [ ]:
dataset_survival['Avg_EMS_group6'] = dataset_survival['Avg_EMS'].apply(Avg_EMS_group)

In [ ]:
dataset_survival.Avg_EMS_group6.value_counts()

In [ ]:
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group6'].unique(): 
    ix = dataset_survival['Avg_EMS_group6'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('KPM survival curve among kidney transplant patients stratified by age groups - Deceased Donors (in days)')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group6 ==  "<= 5.0"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group6 ==  ">5.0"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

###### 2.7, 2.8,3.0  significant 4.0 looks best cut off, 1.1 second place ,check for deceased

### EPLET LOAD

In [ ]:
dataset_survival['Avg_EMS'].unique()

In [ ]:
print(dataset_survival.columns)

In [ ]:
#10 used as cut off
def eplet_group(age):
    if age <= 10: 
        return "<= 10"
    else:
        return ">10"

In [ ]:
dataset_survival['Avg_EMS_group'] = dataset_survival['Avg_EMS'].apply(eplet_group)

In [ ]:
dataset_survival.Avg_EMS_group.value_counts()

In [ ]:
#KPM Curve by AbvMismatch group
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group'].unique():
    ix = dataset_survival['Avg_EMS_group'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('Kaplan-Meier survival curve of overall survival (in days) among kidney transplant stratified by AbvMismatch- Deceased Donors')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group ==  "<= 10"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group ==  ">10"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)


In [ ]:
# print(S1.columns)
# print(S2.columns)


In [ ]:
S1 = S1.dropna(subset=['DAYS', 'GL'])
S2 = S2.dropna(subset=['DAYS', 'GL'])


In [ ]:
S1['GL'] = S1['GL'].astype(int)
S2['GL'] = S2['GL'].astype(int)


In [ ]:
from lifelines.statistics import logrank_test

def logrank_p_value(group_1_data, group_2_data):
    # Ensure variables exist
    print("Checking group_1_data:\n", group_1_data.head())
    print("Checking group_2_data:\n", group_2_data.head())

    # Perform log-rank test
    result = logrank_test(
        group_1_data['DAYS'], group_2_data['DAYS'],
        event_observed_A=group_1_data['GL'], event_observed_B=group_2_data['GL']
    )
    
    return result.p_value


In [ ]:
# print(S1.head())  # Should show a DataFrame with DAYS and GL
# print(S2.head())


In [ ]:
#12 used as cut off

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

# Define a function to categorize Avg_EMS into groups
def eplet_group(value):
    if value <= 12: 
        return "<= 12"
    else:
        return ">12"

# Apply the function to create a new categorical column
dataset_survival['Avg_EMS_group'] = dataset_survival['Avg_EMS'].apply(eplet_group)

# Check the value counts to verify categorization
print(dataset_survival['Avg_EMS_group'].value_counts())

# Kaplan-Meier Curve by Avg_EMS_group
kmf = KaplanMeierFitter()
fig, ax = plt.subplots(figsize=(8,6))  # Create a figure and axis

# Iterate over unique values in the Avg_EMS_group column
for group in dataset_survival['Avg_EMS_group'].unique():
    subset = dataset_survival[dataset_survival['Avg_EMS_group'] == group]
    kmf.fit(subset['DAYS'], event_observed=subset['GL'], label=group)
    kmf.plot(ax=ax, ci_show=False)

# Customize plot labels
plt.xlabel('Time (Days)')
plt.ylabel('Survival Probability Estimate')
plt.title('Kaplan-Meier Survival Curve by Avg_EMS Group (Deceased Donors)')
plt.legend(title='Avg_EMS Group')
plt.show()

# Divide the dataset into two groups
S1 = dataset_survival[dataset_survival['Avg_EMS_group'] == "<= 12"]
S2 = dataset_survival[dataset_survival['Avg_EMS_group'] == ">12"]

# Perform the Log-Rank test
def logrank_p_value(group_1, group_2):
    result = logrank_test(group_1['DAYS'], group_2['DAYS'],
                          event_observed_A=group_1['GL'], event_observed_B=group_2['GL'])
    return result.p_value

p_value = logrank_p_value(S1, S2)
print(f"Log-Rank Test P-Value: {p_value:.4f}")


##### 10 Sig, 12sig best, 14 sig. check for deceased

In [ ]:
# Cut off is 5
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

# Define a function to categorize Avg_EMS into groups based on cutoff 5
def eplet_group_3(value):
    if value <= 5: 
        return "<= 5"
    else:
        return ">5"

# Apply the function to create a new categorical column
dataset_survival['Avg_EMS_group3'] = dataset_survival['Avg_EMS'].apply(eplet_group_3)

# Check the value counts to verify categorization
print(dataset_survival['Avg_EMS_group3'].value_counts())

# Kaplan-Meier Curve by Avg_EMS_group3
kmf = KaplanMeierFitter()
fig, ax = plt.subplots(figsize=(8,6))  # Create a figure and axis

# Iterate over unique values in the Avg_EMS_group3 column
for group in dataset_survival['Avg_EMS_group3'].unique():
    subset = dataset_survival[dataset_survival['Avg_EMS_group3'] == group]
    kmf.fit(subset['DAYS'], event_observed=subset['GL'], label=group)
    kmf.plot(ax=ax, ci_show=False)

# Customize plot labels
plt.xlabel('Time (Days)')
plt.ylabel('Survival Probability Estimate')
plt.title('Kaplan-Meier Survival Curve by Avg_EMS Group3 (Deceased Donors)')
plt.legend(title='Avg_EMS Group3')
plt.show()

# Divide the dataset into two groups
S1 = dataset_survival[dataset_survival['Avg_EMS_group3'] == "<= 5"]
S2 = dataset_survival[dataset_survival['Avg_EMS_group3'] == ">5"]

# Perform the Log-Rank test
def logrank_p_value(group_1, group_2):
    result = logrank_test(group_1['DAYS'], group_2['DAYS'],
                          event_observed_A=group_1['GL'], event_observed_B=group_2['GL'])
    return result.p_value

p_value = logrank_p_value(S1, S2)
print(f"Log-Rank Test P-Value: {p_value:.4f}")


In [ ]:
# used as cut off 6
def eplet_group(age):
    if age <= 6: 
        return "<= 6"
    else:
        return ">6"

In [ ]:
dataset_survival['Avg_EMS_group4'] = dataset_survival['Avg_EMS'].apply(eplet_group)

In [ ]:
dataset_survival.Avg_EMS_group4.value_counts()

In [ ]:
#KPM Curve by AbvMismatch group
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group4'].unique():
    ix = dataset_survival['Avg_EMS_group4'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('Kaplan-Meier survival curve of overall survival (in days) among kidney transplant stratified by Abv Mismatch- Deceased donors')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group4 ==  "<= 6"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group4 ==  ">6"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

In [ ]:
# used as cut off 7
def eplet_group(age):
    if age <= 7: 
        return "<= 7"
    else:
        return ">7"

In [ ]:
dataset_survival['Avg_EMS_group5'] = dataset_survival['Avg_EMS'].apply(eplet_group)

In [ ]:
dataset_survival.Avg_EMS_group5.value_counts()

In [ ]:
#KPM Curve by AbvMismatch group
ax = subplot(111)

for r in dataset_survival['Avg_EMS_group5'].unique():
    ix = dataset_survival['Avg_EMS_group5'] == r
    kmf.fit(T[ix], C[ix], label = r)
    ax=kmf.plot(ax=ax, ci_show=False)
    plt.xlabel('Time (Days)')
    plt.ylabel('Survival probability estimate')
    plt.title('Kaplan-Meier survival curve of overall survival (in days) among kidney transplant stratified by Abv Mismatch- Deceased donors')

In [ ]:
S1 = dataset_survival[dataset_survival.Avg_EMS_group5 ==  "<= 7"]

S2 = dataset_survival[dataset_survival.Avg_EMS_group5 ==  ">7"]

from lifelines.statistics import logrank_test
def logrank_p_value(group_1_data, group_2_data):
    result = logrank_test(group_1_data.DAYS, group_2_data.DAYS,
                          group_1_data.GL, group_2_data.GL)
    return result.p_value

logrank_p_value(S1, S2)

### PREDICTIVE MODELING AND SHAP ANALYSIS

In [ ]:
import xgboost as xgb
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# For modeling, we create a binary target.
# Here we use GL (graft loss) as the event indicator.
dataset_survival['target'] = dataset_survival['GL'].astype(int)

In [ ]:
# Select a set of features (adjust as needed). We include demographic and immunogenicity measures.
features = [
    'REC_AGE_AT_TX',    # Note: if this is categorical, consider encoding appropriately.
    'CAN_GENDER',       # Similarly, encode as needed.
    'DON_RACE',         # May require one-hot encoding.
    'Avg_EMS',
    'Avg_HMS',
    'Avg_AMS'
]

In [ ]:
# Minimal preprocessing: one-hot encode categorical variables.
X = dataset_survival[features].copy()
X = pd.get_dummies(X, drop_first=True)
y = dataset_survival['target']

In [ ]:
# Ensure X is a DataFrame with valid column names
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X)

# Convert feature names to strings and remove invalid characters
X.columns = X.columns.astype(str).str.replace(r"[\[\]<>]", "", regex=True)

# Now proceed with train_test_split and model training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


In [ ]:
# Split into train/test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an XGBoost classifier.
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

In [ ]:
# Evaluate model accuracy.
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
# ---- SHAP Analysis ----
# Use TreeExplainer, which is optimized for tree-based models like XGBoost.
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
# Global feature importance summary plot.
shap.summary_plot(shap_values, X_test)

In [ ]:
# For a local explanation: display a force plot for the first test instance.
shap.initjs()
instance = X_test.iloc[0]
force_plot = shap.force_plot(explainer.expected_value, shap_values[0], instance)
plt.show()

In [ ]:
shap.dependence_plot("Avg_EMS", shap_values, X_test)

In [ ]:
explainer_tree = shap.TreeExplainer(model)
shap_values_tree = explainer_tree.shap_values(X_test)


In [ ]:
# Global explanation: dependence plot for ethnicity.
# Check the name of the ethnicity-related feature. If DON_RACE was one-hot encoded,
# its columns may be like 'DON_RACE_Asian', 'DON_RACE_Black or African American', etc.
# Here, we show a dependence plot for one of them (adjust as needed).
if any("DON_RACE" in col for col in X_test.columns):
    ethnicity_col = [col for col in X_test.columns if "DON_RACE" in col][0]
    shap.dependence_plot(ethnicity_col, shap_values_tree, X_test, show=False)
    plt.title(f"Dependence Plot for {ethnicity_col}")
    plt.show()
else:
    print("No donor race (ethnicity) column found after encoding.")


In [ ]:
# Local explanation: force plot for a selected instance.
shap.initjs()
instance = X_test.iloc[0]
force_plot = shap.force_plot(explainer_tree.expected_value, shap_values_tree[0], instance)
# In a Jupyter Notebook, this will render interactively.
shap.save_html("force_plot_ethnicity.html", force_plot)
print("Force plot saved as 'force_plot_ethnicity.html'.")

In [ ]:
def model_predict(data):
    return model.predict_proba(data)


In [ ]:
background = shap.sample(X_train, 50, random_state=42)  # small sample for background
explainer_kernel = shap.KernelExplainer(model_predict, background)
shap_values_kernel = explainer_kernel.shap_values(X_test.iloc[:10])  # explain a few instances


In [ ]:
print("X_test.iloc[:10].shape:", X_test.iloc[:10].shape)
print("shap_values_kernel[1].shape:", shap_values_kernel[1].shape)


In [ ]:
d = X_test.shape[1]
shap_values_kernel_fixed = shap_values_kernel[1][:, :d]
print("Adjusted SHAP values shape:", shap_values_kernel_fixed.shape)


In [ ]:
background = shap.sample(X_train, 50, random_state=42)
explainer_kernel = shap.KernelExplainer(model_predict, background)
shap_values_kernel = explainer_kernel.shap_values(X_test.iloc[:10])


In [ ]:
single_instance = X_test.iloc[[0]]  # Double brackets to maintain DataFrame format
print("Single instance shape:", single_instance.shape)
print("Single instance content:\n", single_instance)


In [ ]:
explainer = shap.TreeExplainer(model)  # If using tree-based models (e.g., RandomForest, XGBoost)



In [ ]:
# Compute SHAP values for single instance
shap_values_single = explainer.shap_values(single_instance)

if shap_values_single is not None:
    shap_values_single_fixed = shap_values_single
    print("SHAP values shape:", shap_values_single_fixed.shape)
    shap.summary_plot(shap_values_single_fixed, single_instance, show=False)
    plt.title("SHAP Summary Plot for Single Instance (KernelExplainer)")
    plt.show()
else:
    print("Error: SHAP values were not generated correctly.")


In [ ]:
print("KernelExplainer SHAP values shape:", np.array(shap_values_kernel).shape)
print("X_test.iloc[:10] shape:", X_test.iloc[:10].shape)

# Ensure background and X_test are both taken from X_train and X_test (after one-hot encoding)
background = shap.sample(X_train, 50, random_state=42)
explainer_kernel = shap.KernelExplainer(model_predict, background)
shap_values_kernel = explainer_kernel.shap_values(X_test.iloc[:10])

print(f"SHAP values shape: {shap_values_kernel[0].shape if isinstance(shap_values_kernel, list) else shap_values_kernel.shape}")
print(f"X_test shape: {X_test.iloc[:10].shape}")

if isinstance(shap_values_kernel, list):
    print(f"Number of classes in SHAP output: {len(shap_values_kernel)}")
    shap_ethnicity = shap_values_kernel[1]  # Choose the correct class
else:
    shap_ethnicity = shap_values_kernel

print("X_test sample shape:", X_test.iloc[:10].shape)
print("SHAP values shape:", shap_ethnicity.shape)

shap.dependence_plot(ethnicity_col, shap_values_kernel[:, :, 1], X_test.iloc[:10], show=False)  # FIXED LINE
plt.title(f"KernelExplainer: Dependence Plot for {ethnicity_col}")
plt.show()



In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Columns in X_test:", X_test.columns.tolist())


In [ ]:
# Make sure to use the same processed X_train and X_test
background = shap.sample(X_train, 50, random_state=42)
def model_predict(data):
    return model.predict_proba(data)
explainer_kernel = shap.KernelExplainer(model_predict, background)
shap_values_kernel = explainer_kernel.shap_values(X_test.iloc[:10])
print("KernelExplainer SHAP values shape:", np.array(shap_values_kernel).shape)


In [ ]:
if isinstance(shap_values_kernel, list):
    print("Shape for class 0:", shap_values_kernel[0].shape)
    print("Shape for class 1:", shap_values_kernel[1].shape)
else:
    print("Shape:", np.array(shap_values_kernel).shape)


In [ ]:
single_instance = X_test.iloc[[0]]
shap_values_single = explainer_kernel.shap_values(single_instance)
if isinstance(shap_values_single, list):
    # If a list is returned, choose the positive class if available
    shap_values_single_used = shap_values_single[1]
else:
    shap_values_single_used = shap_values_single
print("Single instance data shape:", single_instance.shape)
print("Single instance SHAP values shape:", shap_values_single_used.shape)
print("Feature names:", single_instance.columns.tolist())


In [ ]:
# ---- Interpreting Survival Outcomes via SHAP ----
# Although our model is for predicting graft loss, you can relate the SHAP outputs to overall kidney graft allocation survival.
# For example, if higher predicted probability corresponds to poorer survival, then the SHAP contributions
# (especially for ethnicity) can be interpreted in that context.
# You might overlay the SHAP values on Kaplan-Meier curves if desired.
#
# One simple approach is to color-code Kaplan-Meier plots by predicted risk.
# (Below is a conceptual example; you might refine it for publication.)

from lifelines import KaplanMeierFitter

In [ ]:
# Compute predicted probabilities for graft loss.
dataset_survival['pred_risk'] = model.predict_proba(X)[:, 1]

In [ ]:
# Create a risk group based on median predicted risk.
median_risk = dataset_survival['pred_risk'].median()
dataset_survival['risk_group'] = dataset_survival['pred_risk'].apply(lambda x: 'High Risk' if x > median_risk else 'Low Risk')

In [ ]:
# Plot Kaplan-Meier curves by risk group.
kmf_high = KaplanMeierFitter()
kmf_low = KaplanMeierFitter()

In [ ]:
T = dataset_survival['DAYS']
C = dataset_survival['GL']

In [ ]:
high_risk = dataset_survival[dataset_survival['risk_group'] == 'High Risk']
low_risk = dataset_survival[dataset_survival['risk_group'] == 'Low Risk']

In [ ]:
plt.figure(figsize=(10,6))
kmf_high.fit(high_risk['DAYS'], event_observed=high_risk['GL'], label='High Predicted Risk')
ax = kmf_high.plot(ci_show=False)
kmf_low.fit(low_risk['DAYS'], event_observed=low_risk['GL'], label='Low Predicted Risk')
kmf_low.plot(ax=ax, ci_show=False)
plt.xlabel("Time (Days)")
plt.ylabel("Survival Probability")
plt.title("Kaplan-Meier Survival Curves by Predicted Graft Loss Risk")
plt.show()

In [ ]:
print(type(shap_values_tree))  # Should print <class 'list'> if multi-class
if isinstance(shap_values_tree, list):
    for i, arr in enumerate(shap_values_tree):
        print(f"Class {i} SHAP values shape:", arr.shape)
else:
    print("SHAP values shape:", shap_values_tree.shape)


In [ ]:
ethnicity_cols = [i for i, col in enumerate(X_test.columns) if "DON_RACE" in col]

if len(ethnicity_cols) > 0:
    # Extract SHAP values for ethnicity features
    ethnicity_shap = shap_values_tree[:, ethnicity_cols]  # Use indices instead of column names
    
    # Compute average absolute SHAP values per ethnicity feature
    avg_shap = np.abs(ethnicity_shap).mean(axis=0)
    
    print("Average absolute SHAP values for ethnicity features:")
    for feature, value in zip([X_test.columns[i] for i in ethnicity_cols], avg_shap):
        print(f"{feature}: {value:.3f}")
else:
    print("No ethnicity features found for SHAP analysis.")
